In [3]:
import transformer_lens

# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("google/gemma-2-2b", cache_dir=)

model-00006-of-00008.safetensors:  25%|##4       | 1.22G/4.96G [00:00<?, ?B/s]

e:\anaconda3\envs\sae\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\嘉庆\.cache\huggingface\hub\models--google--gemma-2-9b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

: 

In [ ]:
prompt = "Would you be able to travel through time using a wormhole?"
# Use the tokenizer to convert it to tokens. Note that this implicitly adds a special "Beginning of Sequence" or <bos> token to the start
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to(device)
print(inputs)
_, cache = model.run_with_cache(inputs)
# %%
target_act = cache[f'blocks.{layer_idx}.hook_resid_post'].squeeze()
# target_act = cache[f'hook_embed'].squeeze()
sae_acts = sae.encode(target_act.to(torch.float32))
recon = sae.decode(sae_acts)
1 - torch.mean((recon[1:, :] - target_act[1:, :].to(torch.float32)) **2) / (target_act[1:, :].to(torch.float32).var())
# %%
(sae_acts > 1).sum(-1)
# %%
values, inds = sae_acts.max(-1)
inds